In [1]:
from database.sec import SEC
from database.market import Market
from transformer.sec_transformer import SECTransformer
from processor.processor import Processor as p
from datetime import datetime
from tqdm import tqdm

In [2]:
sec = SEC()
market = Market()

In [3]:
sec.connect()
year = 2023
quarter = 3
sub = SECTransformer.transform(year,quarter,"sub")[["adsh","cik","filed"]]
num = SECTransformer.transform(year,quarter,"num")
for adsh in num["adsh"].unique():
    try:
        filing = num[num["adsh"]==adsh].pivot_table(index="adsh",columns="tag",values="value").reset_index()
        for column in filing.columns:
            if "." in column:
                new = column.replace(".","-")
                filing.rename(columns={column:new},inplace=True)
        complete = p.column_date_processing(filing)
        complete = complete.merge(sub,on="adsh",how="left")
        sec.store("filings",complete)
    except Exception as e:
        print(adsh,str(e))
sec.disconnect()

  0%|                                                                                                                                                                                   | 0/12 [00:00<?, ?it/s]E:\projects\longshot_venv\longshot\database\adatabase.py:40: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  records = data.to_dict("records")
 17%|███████████████████████████▎                                                                                                                                        | 2/12 [3:32:38<17:56:33, 6459.40s/it]E:\projects\longshot_venv\longshot\database\adatabase.py:40: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  records = data.to_dict("records")
 75%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                         | 9/12 [13:26:05<3:56:47, 4735.86s/it]

sec filings documents must be a non-empty list


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [17:34:44<00:00, 5273.70s/it]

2023 3 [Errno 2] No such file or directory: './sec/2023q3/sub.txt'
2023 4 [Errno 2] No such file or directory: './sec/2023q4/sub.txt'
